In [1]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [2]:
data_dir = "../data"
infantNum = 6
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')

ECG DATA: {'fs': 500, 'sig_len': 87492197, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['II'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 8749057, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
174984.394 sec, 2916.4065666666665 minutes, 48.60677611111111 hours
174981.14 sec, 2916.3523333333337 minutes, 48.605872222222224 hours


In [3]:
# initialize the training
start_time_secs = hours2secs(2)
end_time_secs = hours2secs(2.2)
lr_model, pr_model, svr_rbf_model = training_pipeline(start_time_secs=start_time_secs, 
                                                        end_time_secs=end_time_secs, 
                                                        signal_ecg_0=signal_ecg_0,
                                                        signal_resp_0=signal_resp_0,
                                                        ECG_dataset=ECG_dataset,
                                                        RESP_dataset=RESP_dataset)

start: 3600000, end: 3960000
start: 360000, end: 396000
90th percentile: 0.05495781300878913, 10th percentile: -0.1473868621599345, IQR: 0.20234467516872362
Identified outliers: 24745
Data Shape: (360000,)
90th percentile: 21.237561922752704, 10th percentile: 19.207829213714092, IQR: 2.029732709038612
Identified outliers: 1040
Data Shape: (36000,)
[[149.68919008   9.61437089]
 [  9.61437089 102.58690915]]
Pearsons correlation: 0.078
Spearmans correlation: 0.041
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 0.006297841564065432
Intercept: [139.62411]
Coefficients: [[0.06586734]]
Mean absolute error: 4.55
Mean squared error: 100.61
Root mean squared error: 10.03
---------------------------------------------
--- Polynomial Regression ---
Coefficient of determination: 0.011452922932024157
Intercept: [140.55786]
Coefficients: [[ 7.3383949e-10  1.6478767e-08  6.7856877e-07  1.0872394e-05
  -4.5145441e-07  4.7800359e-09]]
Mean absolute e

In [4]:
step = 0.25
startHrs = 10
endHrs = 12
logFilePath = f"../logs/infant{infantNum}_step{step}_startHrs{startHrs}_endHrs{endHrs}.log"

with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(startHrs, endHrs, step):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.3f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+step)
            lr_model, pr_model, svr_rbf_model = cascade_training_pipeline(lr_model=lr_model, 
                                                                            pr_model=pr_model, 
                                                                            svr_rbf_model=svr_rbf_model, 
                                                                            start_time_secs=start_time_secs, 
                                                                            end_time_secs=end_time_secs,
                                                                            signal_ecg_0=signal_ecg_0,
                                                                            signal_resp_0=signal_resp_0,
                                                                            ECG_dataset=ECG_dataset,
                                                                            RESP_dataset=RESP_dataset)

In [5]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model]:
    filename = f"../models/infant_{infantNum}/{namestr(i, globals())[0]}_step{step}_startHrs{startHrs}_endHrs{endHrs}.sav"
    joblib.dump(i, filename)
